# Example

In [74]:
import pandas as pd
import numpy as np

## Data Sources

* IEA PVPS T12
* EXIOBASE v3 Hybrid (EXIO3h)

## Sanitising input data

### IEA PVPS T12 data

Worked backwards from requirements for PV panel single Si, produced in CN. Categorised separate LCI elements into:
* products: the product produced by the unit process,
* components: elements required to produce the product that have their own LCI sheet,
* resources: elements required to produce the product that do not have their own sheet and hence need to be connected to the hIOT,
* emissions: direct emissions of the process, limited to "Carbon dioxide, fossil" to air.

## Loading input data

### Load unit process data

In [1]:
# load the LCI data from the Excel file
with pd.ExcelFile("PV_LCIs_final_China.xlsx") as LCIs_source:
    LCI_names = LCIs_source.sheet_names
    LCIs = [pd.read_excel(LCIs_source, sn) for sn in LCI_names]

LCIs[0]

NameError: name 'pd' is not defined

### Load EXIO3h data

In [3]:
# load the full inter-industry matrix Z
Z = pd.read_csv("MR_HIOT_2011_v3_3_18_by_product_technology.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2, 3, 4])
Z.index = Z.index.droplevel([2, 3])
Z.columns = Z.columns.droplevel([2, 3]) # remove codes, keep full descriptions (should probably keep codes instead)

# load the final demand vector y
y = pd.read_csv(
    "MR_HIOT_2011_v3_3_18_FD.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2, 3, 4])
y.index = y.index.droplevel([2, 3])
y.columns = y.columns.droplevel([2, 3])
    # sum this to a single vector?

# load the emissions vector B
B = pd.read_excel(
    "MR_HIOT_2011_v3_3_18_extensions.xlsx",
    sheet_name="Emiss_act",
    header=[0, 1, 2, 3],
    index_col=[0, 1, 2],
).loc["Carbon dioxide, fossil"]     # keep only CO2 to air emissions

B.columns = B.columns.droplevel([2, 3])

B = pd.concat([B], keys=["Carbon dioxide, fossil"])

In [4]:
x_full = pd.read_csv("MR_HIOT_2011_v3_3_18_principal_production.csv", header=[0, 1, 2, 3, 4, 5, 6, 7])

x = x_full.copy()
x.columns = x.columns.droplevel([2, 3, 4, 5, 6, 7])
x = x.T[0]
x

xProd = x_full.copy()
xProd.columns = xProd.columns.droplevel([1, 2, 3, 5, 6])
xProd = xProd.T[0]
xProd

AU  Paddy rice                                         tonnes    6.147906e+05
    Wheat                                              tonnes    2.329856e+07
    Cereal grains nec                                  tonnes    1.009107e+07
    Vegetables, fruit, nuts                            tonnes    3.256923e+06
    Oil seeds                                          tonnes    1.054723e+06
                                                                     ...     
WM  Membership organisation services n.e.c. (91)       Meuro     1.549903e+04
    Recreational, cultural and sporting services (92)  Meuro     3.103584e+04
    Other services (93)                                Meuro     2.187647e+04
    Private households with employed persons (95)      Meuro     5.306091e+03
    Extra-territorial organizations and bodies         Meuro     0.000000e+00
Name: 0, Length: 7872, dtype: float64

In [5]:
# compute the technology matrix A
A = Z.div(x, axis=1).fillna(0)

## Creating the full table structure and filling in the hIOT

### Get the unit processes from the LCI data.

In [6]:
# get all the "product" rows (i.e. unit processes): these form extra rows in the eventual table
LCI_rows = pd.concat([df.query("type == 'product'") for df in LCIs]).dropna(axis=1, how="all").drop(["type", "value"], axis=1)
LCI_rows = LCI_rows[["location", "description", "unit"]]    # reorder to match hIOT order

# create index from LCI unit processes to merge with hIOT index
LCI_index = pd.MultiIndex.from_frame(LCI_rows)

### Create the full table structure

In [7]:
full_index = LCI_index.union(A.index, sort=False)   # LCI indices top and left, hIOT indices bottom and right
full_columns = (LCI_index
                    .droplevel(2)   # drop units from LCI index (no units in hIOT columns)
                    .union(A.columns, sort=False)   # append columns of A but keep order
)

full_table = pd.DataFrame(np.zeros((len(full_index), len(full_index))), index=full_index, columns=full_columns) # full table of zeros

display(full_table.head())
display(full_table.tail())

CN  \
                                                         photovoltaic panel, single-Si, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                     0.0   
   photovoltaic cell, single-Si, at plant             m2                                     0.0   
   single-Si wafer, photovoltaics, at plant           m2                                     0.0   
   CZ single crystalline silicon, photovoltaics, a... kg                                     0.0   
   silicon, production mix, photovoltaics, at plant   kg                                     0.0   

                                                                                                 \
                                                         photovoltaic cell, single-Si, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                    0.0   
   photovoltaic cell, single-Si, at plant             m2                                    0.0   
   single-Si wafer, photovoltaics, at plant           m2                                    0.0   
   CZ single crystalline silicon, photovoltaics, a... kg                                    0.0   
   silicon, production mix, photovoltaics, at plant   kg                                    0.0   

                                                                                                   \
                                                         single-Si wafer, photovoltaics, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                      0.0   
   photovoltaic cell, single-Si, at plant             m2                                      0.0   
   single-Si wafer, photovoltaics, at plant           m2                                      0.0   
   CZ single crystalline silicon, photovoltaics, a... kg                                      0.0   
   silicon, production mix, photovoltaics, at plant   kg                                      0.0   

                                                                                                                 \
                                                         CZ single crystalline silicon, photovoltaics, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                                0.0       
   photovoltaic cell, single-Si, at plant             m2                                                0.0       
   single-Si wafer, photovoltaics, at plant           m2                                                0.0       
   CZ single crystalline silicon, photovoltaics, a... kg                                                0.0       
   silicon, production mix, photovoltaics, at plant   kg                                                0.0       

                                                                                                           \
                                                         silicon, production mix, photovoltaics, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                              0.0   
   photovoltaic cell, single-Si, at plant             m2                                              0.0   
   single-Si wafer, photovoltaics, at plant           m2                                              0.0   
   CZ single crystalline silicon, photovoltaics, a... kg                                              0.0   
   silicon, production mix, photovoltaics, at plant   kg                                              0.0   

                                                                                                              RER  \
                                                         silicon, solar grade, modified Siemens process, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                                0.0         
   photovoltaic cell, single-Si, at plant             m2              

CN  \
                                                           photovoltaic panel, single-Si, at plant   
WM Membership organisation services n.e.c. (91)      Meuro                                     0.0   
   Recreational, cultural and sporting services (92) Meuro                                     0.0   
   Other services (93)                               Meuro                                     0.0   
   Private households with employed persons (95)     Meuro                                     0.0   
   Extra-territorial organizations and bodies        Meuro                                     0.0   

                                                                                                   \
                                                           photovoltaic cell, single-Si, at plant   
WM Membership organisation services n.e.c. (91)      Meuro                                    0.0   
   Recreational, cultural and sporting services (92) Meuro                                    0.0   
   Other services (93)                               Meuro                                    0.0   
   Private households with employed persons (95)     Meuro                                    0.0   
   Extra-territorial organizations and bodies        Meuro                                    0.0   

                                                                                                     \
                                                           single-Si wafer, photovoltaics, at plant   
WM Membership organisation services n.e.c. (91)      Meuro                                      0.0   
   Recreational, cultural and sporting services (92) Meuro                                      0.0   
   Other services (93)                               Meuro                                      0.0   
   Private households with employed persons (95)     Meuro                                      0.0   
   Extra-territorial organizations and bodies        Meuro                                      0.0   

                                                                                                                   \
                                                           CZ single crystalline silicon, photovoltaics, at plant   
WM Membership organisation services n.e.c. (91)      Meuro                                                0.0       
   Recreational, cultural and sporting services (92) Meuro                                                0.0       
   Other services (93)                               Meuro                                                0.0       
   Private households with employed persons (95)     Meuro                                                0.0       
   Extra-territorial organizations and bodies        Meuro                                                0.0       

                                                                                                             \
                                                           silicon, production mix, photovoltaics, at plant   
WM Membership organisation services n.e.c. (91)      Meuro                                              0.0   
   Recreational, cultural and sporting services (92) Meuro                                              0.0   
   Other services (93)                               Meuro                                              0.0   
   Private households with employed persons (95)     Meuro                                              0.0   
   Extra-territorial organizations and bodies        Meuro                                              0.0   

                                                                                                                RER  \
                                                           silicon, solar grade, modified Siemens process, at plant   
WM Membership organisation services n.e.c. (91)      Meuro                                                0.0      

### Fill in the elements of $I - A$ in the bottom-right block of the full table

In [8]:
full_table.loc[A.index, A.columns] = np.eye(A.shape[0]) - A.values  # fill in hIOT elements with I-A (bottom-right block)

## Convert LCIs to vectors in format of full table

### Load conversion table
We use a manually created conversion table here, but this should be automatised.

In [97]:
conv_table = pd.read_excel("background_table_construction.xlsx", sheet_name="All entries").drop(["Sheet", "Type", "EXIO location"], axis=1)
conv_table.head()

conv_table[conv_table["EXIO3 unit"] == "TJ"]

,Entry,Location,Unit,EXIO3 entry,EXIO3 product code,EXIO3 unit,Conversion Factor
6,"electricity, medium voltage, at grid",CN,kWh,Electricity nec,C_POWZ,TJ,277777.778
7,"electricity, medium voltage, at grid",US,kWh,Electricity nec,C_POWZ,TJ,277777.778
8,"electricity, medium voltage, at grid",KR,kWh,Electricity nec,C_POWZ,TJ,277777.778
9,"electricity, medium voltage, production ENTSO,...",ENTSO,kWh,Electricity nec,C_POWZ,TJ,277777.778
35,"electricity, medium voltage, at grid",NO,kWh,Electricity nec,C_POWZ,TJ,277777.778
36,"electricity, medium voltage, at grid",CN,kWh,Electricity nec,C_POWZ,TJ,277777.778
37,"electricity, medium voltage, at grid",US,kWh,Electricity nec,C_POWZ,TJ,277777.778
38,"electricity, medium voltage, at grid",KR,kWh,Electricity nec,C_POWZ,TJ,277777.778
51,"electricity, medium voltage, at grid",CN,kWh,Electricity nec,C_POWZ,TJ,277777.778
52,"electricity, medium voltage, at grid",US,kWh,Electricity nec,C_POWZ,TJ,277777.778


### Convert descriptions, units and values

In [116]:
avs = []
ems = []

for lci in LCIs:
    df = lci.copy()

    df = df.merge(right=conv_table, left_on="description", right_on="Entry", how="left")
    res_mask = df["type"] == "resources"

    df.loc[res_mask, "description"] = df.loc[res_mask, "EXIO3 entry"]
    df.loc[res_mask, "value"] = df.loc[res_mask, "value"] / df.loc[res_mask, "Conversion Factor "]
    df.loc[res_mask, "unit"] = df.loc[res_mask, "EXIO3 unit"]

    full_vector = df.loc[df["type"] != "emissions", ["location", "description", "unit", "value"]]

    agg_vector = full_vector.set_index(["location", "description", "unit"])
    avs.append(agg_vector)

In [122]:
avs[3].query("unit == 'TJ'")

value
location description     unit          
CN       Electricity nec TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115
                         TJ    0.000115

### Create distribution vectors

Since some locations are listed as "RER" or "GLO", we need to distribute these over the locations available. We do this by full production ratio.

In [99]:
def get_distribution_vector(product, region):
    if region == "GLO":
        countries = Z.index.get_level_values(0)
    elif region == "RER":
        countries = ["AT", "BE", "BG", "CZ", "DE", "DK", "EE", "ES", "FI", "FR", "GR",
                     "HR", "HU", "IE", "IT", "LT", "LU", "LV", "MT", "NL", "PL", "PT", 
                     "RO", "SE", "SI", "SK", "GB", "RU", "CH", "NO"]
    elif region == "OCE":
        countries = ["AU", "ID"]

    production_vector = xProd.loc[(countries, product)]
    # production_vector = Z.loc[(countries, product, slice(None), slice(None))]
    parts = production_vector / production_vector.sum()
    return parts

get_distribution_vector("Aluminium and aluminium products", "RER")    # example

AT  Aluminium and aluminium products  tonnes    0.000000
BE  Aluminium and aluminium products  tonnes    0.000000
BG  Aluminium and aluminium products  tonnes    0.000000
CZ  Aluminium and aluminium products  tonnes    0.000000
DE  Aluminium and aluminium products  tonnes    0.055681
DK  Aluminium and aluminium products  tonnes    0.000000
EE  Aluminium and aluminium products  tonnes    0.000000
ES  Aluminium and aluminium products  tonnes    0.052582
FI  Aluminium and aluminium products  tonnes    0.000000
FR  Aluminium and aluminium products  tonnes    0.043003
GR  Aluminium and aluminium products  tonnes    0.021263
HR  Aluminium and aluminium products  tonnes    0.000000
HU  Aluminium and aluminium products  tonnes    0.003632
IE  Aluminium and aluminium products  tonnes    0.000000
IT  Aluminium and aluminium products  tonnes    0.018270
LT  Aluminium and aluminium products  tonnes    0.000000
LU  Aluminium and aluminium products  tonnes    0.000000
LV  Aluminium and aluminium pro

## Creating the LCI vectors and the full table

### Disaggregating the regions

In [115]:
LCI_vectors = []

for av in avs:
    agg_vector = av.reset_index()

    is_resource = agg_vector["description"].isin(Z.index.get_level_values(1))
    is_aggregated = agg_vector["location"].isin(["RER", "GLO", "OCE"])

    to_disaggregate = is_resource & is_aggregated
    no_disaggregation = agg_vector[~to_disaggregate].set_index(["location", "description", "unit"])

    disaggregated_vector = agg_vector.loc[to_disaggregate].T.apply(lambda s: s["value"] * get_distribution_vector(s["description"], s["location"])).sum(1)
    disaggregated_vector.name = "value"
    LCI_vector = pd.concat([no_disaggregation, disaggregated_vector.to_frame()])
    LCI_vectors.append(LCI_vector)

### Creating a dataframe with one column per LCI

In [114]:
full_lci_list = []

for LCIv in LCI_vectors:
    lci = LCIv["value"]
    full_lci = pd.Series(index=full_table.index)
    full_lci[lci.index] = lci

    full_lci_list.append(full_lci)

LCI_block = pd.concat(full_lci_list, axis=1).fillna(0)
LCI_block.columns = full_table.columns[:len(LCI_vectors)]

LCI_block.head()

CN  \
                                                         photovoltaic panel, single-Si, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                1.000000   
   photovoltaic cell, single-Si, at plant             m2                                0.935415   
   single-Si wafer, photovoltaics, at plant           m2                                0.000000   
   CZ single crystalline silicon, photovoltaics, a... kg                                0.000000   
   silicon, production mix, photovoltaics, at plant   kg                                0.000000   

                                                                                                 \
                                                         photovoltaic cell, single-Si, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                   0.00   
   photovoltaic cell, single-Si, at plant             m2                                   1.00   
   single-Si wafer, photovoltaics, at plant           m2                                   1.03   
   CZ single crystalline silicon, photovoltaics, a... kg                                   0.00   
   silicon, production mix, photovoltaics, at plant   kg                                   0.00   

                                                                                                   \
                                                         single-Si wafer, photovoltaics, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                 0.000000   
   photovoltaic cell, single-Si, at plant             m2                                 0.000000   
   single-Si wafer, photovoltaics, at plant           m2                                 1.000000   
   CZ single crystalline silicon, photovoltaics, a... kg                                 0.595238   
   silicon, production mix, photovoltaics, at plant   kg                                 0.000000   

                                                                                                                 \
                                                         CZ single crystalline silicon, photovoltaics, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                                0.0       
   photovoltaic cell, single-Si, at plant             m2                                                0.0       
   single-Si wafer, photovoltaics, at plant           m2                                                0.0       
   CZ single crystalline silicon, photovoltaics, a... kg                                                1.0       
   silicon, production mix, photovoltaics, at plant   kg                                                1.0       

                                                                                                           \
                                                         silicon, production mix, photovoltaics, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                              0.0   
   photovoltaic cell, single-Si, at plant             m2                                              0.0   
   single-Si wafer, photovoltaics, at plant           m2                                              0.0   
   CZ single crystalline silicon, photovoltaics, a... kg                                              0.0   
   silicon, production mix, photovoltaics, at plant   kg                                              1.0   

                                                                                                              RER  \
                                                         silicon, solar grade, modified Siemens process, at plant   
CN photovoltaic panel, single-Si, at plant            m2                                                0.0         
   photovoltaic cell, single-Si, at plant             m2              

### Putting the LCI block in the right format
To get the LCI block into the format of a technology matrix $T$, we need to multiply each off-diagonal element by -1.

In [102]:
LCI_block = LCI_block * -1

for i in range(len(LCI_vectors)):
    LCI_block.iloc[i, i] = 1

### Entering the LCI block in the full table

In [103]:
full_table.loc[LCI_block.index, LCI_block.columns] = LCI_block
full_table = full_table.fillna(0)
full_table.shape

(7885, 7885)

## Creating the emissions vector

The vector from the hIOT extensions:

In [104]:
B_norm = B.div(x)

### Creating the emissions vector from the LCIs

In [18]:
em_values = pd.concat([lci[lci["type"] == "emissions"].reset_index()["value"] for lci in LCIs], axis=1)
em_values = em_values / 1000
em_values.fillna(0)

em_values.index = B.index
em_values.columns = full_table.columns[:len(LCIs)]

full_emissions_vector = pd.concat([em_values, B_norm], axis=1)
full_emissions_vector = full_emissions_vector.fillna(0)

full_emissions_vector

CN  \
                                  photovoltaic panel, single-Si, at plant   
Carbon dioxide, fossil tonnes air                                0.000022   

                                                                          \
                                  photovoltaic cell, single-Si, at plant   
Carbon dioxide, fossil tonnes air                               0.000167   

                                                                            \
                                  single-Si wafer, photovoltaics, at plant   
Carbon dioxide, fossil tonnes air                                      0.0   

                                                                                          \
                                  CZ single crystalline silicon, photovoltaics, at plant   
Carbon dioxide, fossil tonnes air                                                0.0       

                                                                                    \
                                  silicon, production mix, photovoltaics, at plant   
Carbon dioxide, fossil tonnes air                                              0.0   

                                                                                       RER  \
                                  silicon, solar grade, modified Siemens process, at plant   
Carbon dioxide, fossil tonnes air                                                0.0         

                                                                                        CN  \
                                  silicon, solar grade, modified Siemens process, at plant   
Carbon dioxide, fossil tonnes air                                                0.0         

                                                                                        US  \
                                  silicon, solar grade, modified Siemens process, at plant   
Carbon dioxide, fossil tonnes air                                                0.0         

                                                                                      APAC  \
                                  silicon, solar grade, modified Siemens process, at plant   
Carbon dioxide, fossil tonnes air                                                0.0         

                                                    NO  ...  \
                                  MG-silicon, at plant  ...   
Carbon dioxide, fossil tonnes air             0.003581  ...   

                                                        WM  \
                                  Landfill of waste: Paper   
Carbon dioxide, fossil tonnes air                 0.000015   

                                                              \
                                  Landfill of waste: Plastic   
Carbon dioxide, fossil tonnes air                    0.01337   

                                                                            \
                                  Landfill of waste: Inert/metal/hazardous   
Carbon dioxide, fossil tonnes air                                 0.002092   

                                                               \
                                  Landfill of waste: Textiles   
Carbon dioxide, fossil tonnes air                    0.113862   

                                                           \
                                  Landfill of waste: Wood   
Carbon dioxide, fossil tonnes air                0.000015   

                                                                                     \
                                  Activities of membership organisation n.e.c. (91)   
Carbon dioxide, fossil tonnes air                                        490.406924   

                                                                                       \
                                  Recreational, cultural and sporting activities (92)   
Carbon dioxide, fossil tonnes air                                         208.9439

### Creating a single final demand vector

In [105]:
y1 = np.zeros(full_table.shape[1])
y1[0] = 1

FD = pd.DataFrame(y1, index=full_table.index, columns=["final demand"])

## Calculate the full impacts

In [106]:
L = pd.DataFrame(np.linalg.inv(full_table.values), index=full_table.index, columns=full_table.columns)

In [107]:
divs = pd.DataFrame(full_emissions_vector.values @ L.values, columns=full_table.index)

In [108]:
divs @ y1

0    2.073961
dtype: float64

In [109]:
L.iloc[:, 8].sort_values()

WA    products of Vegetable oils and fats                                                                     tonnes   -0.000140
      Oil seeds                                                                                               tonnes   -0.000112
CN    Cement, lime and plaster                                                                                tonnes   -0.000095
FR    products of Vegetable oils and fats                                                                     tonnes   -0.000088
ID    products of Vegetable oils and fats                                                                     tonnes   -0.000080
                                                                                                                          ...   
RU    Natural gas and services related to natural gas extraction, excluding surveying; inclulding liquid gas  tonnes    0.003290
CN    Coal, lignite and peat                                                                     

In [26]:
result = pd.Series(np.diag(full_emissions_vector.values[0]) @ L.iloc[:, 0].values, index=L.index).sort_values(ascending=False)

In [27]:
result

CN  Electricity by coal                                                  TJ        1.254316
    Basic iron and steel and of ferro-alloys and first products thereof  tonnes    0.069604
    Collected and purified water, distribution services of water (41)    Meuro     0.068741
    Steam and hot water supply services                                  TJ        0.063057
RU  Chemicals nec; additives and biofuels                                tonnes    0.030142
                                                                                     ...   
WA  products of Vegetable oils and fats                                  tonnes   -0.000091
ID  Oil seeds                                                            tonnes   -0.000114
WA  Oil seeds                                                            tonnes   -0.000151
GR  Cement, lime and plaster                                             tonnes   -0.000204
CN  Cement, lime and plaster                                             tonnes 

In [110]:
pd.Series((full_emissions_vector.values @ L.values)[0], index=L.index).values * full_table.iloc[:, 7].sort_values()

US  MG-silicon, at plant                                      kg       -2.343576
RU  Chemicals nec; additives and biofuels                     tonnes   -0.004066
DE  Chemicals nec; additives and biofuels                     tonnes   -0.003461
ES  Chemicals nec; additives and biofuels                     tonnes   -0.000924
PL  Chemicals nec; additives and biofuels                     tonnes   -0.000159
                                                                          ...   
HR  Raw milk                                                  tonnes   -0.000000
    Animal products nec                                       tonnes   -0.000000
    Copper ores and concentrates                              tonnes   -0.000000
WM  Extra-territorial organizations and bodies                Meuro    -0.000000
US  silicon, solar grade, modified Siemens process, at plant  kg        0.000000
Name: (US, silicon, solar grade, modified Siemens process, at plant), Length: 7885, dtype: float64

In [111]:
full_table.iloc[:, 0].sort_values(ascending=True)

CN  photovoltaic cell, single-Si, at plant                                          m2                 -0.935415
DE  Glass and glass products                                                        tonnes             -0.001776
RU  Chemicals nec; additives and biofuels                                           tonnes             -0.001410
DE  Chemicals nec; additives and biofuels                                           tonnes             -0.001282
ES  Chemicals nec; additives and biofuels                                           tonnes             -0.001212
                                                                                                          ...   
HR  Ceramic goods                                                                   tonnes             -0.000000
    Secondary glass for treatment, Re-processing of secondary glass into new glass  tonnes (service)   -0.000000
    P- and other fertiliser                                                         tonnes      